# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# NetCDF Output

In [ ]:
import pathlib as pl

import flopy
import matplotlib.pyplot as plt
import xarray as xa
import xugrid as xu

## Load the base watershed model

We will use the base watershed model to demonstrate MODFLOW NetCDF output functionality.  

In [ ]:
name = "watershed"
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=pl.Path("../../data/watershed/")
)

## Modify the model to save structured NetCDF output

We need to set the `start_date_time` variable in the TDIS file and define the `export_netcdf` variable in the groundwater flow model `name_file`. We will create `structured` NetCDF output first. 

In [ ]:
sim.tdis.start_date_time = "1980-01-01"

In [ ]:
gwf = sim.get_model()
gwf.name_file.export_netcdf = "structured"

### Change the simulation workspace, write the model files, and run the model

In [ ]:
ws = pl.Path("temp/structured")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### Load the structured NetCDF file with `xarray` and plot the results

In [ ]:
pl.Path(ws / "model.nc").is_file()

In [ ]:
nc_ds = xa.open_dataset(ws / "model.nc")

In [ ]:
nc_ds

In [ ]:
nc_ds["head"].isel(time=0, z=0).plot()
plt.gca().set_aspect(1.0)
plt.show(block=False)

## Modify the model to save mesh NetCDF output

We need modufy the `export_netcdf` variable in the groundwater flow model `name_file` to `ugrid` to create mesh NetCDF output.

In [ ]:
gwf.name_file.export_netcdf = "ugrid"

### Change the simulation workspace, write the model files, and run the model

In [ ]:
ws = pl.Path("temp/mesh")
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### Load the mesh NetCDF file with `xugrid` and plot the results

In [ ]:
nc_ds = xu.open_dataset(ws / "model.nc")

In [ ]:
nc_ds

In [ ]:
nc_ds["head_l1"].isel(time=0).ugrid.plot()
plt.gca().set_aspect(1.0)
plt.show(block=False)

### Create cross-sections using `xugrid`

First, we will create a cross-section along a column (y-direction).

In [ ]:
section = nc_ds.ugrid.sel(x=101250)

In [ ]:
section

In [ ]:
section["head_l1"].isel(time=0).plot.line(x="mesh_y")  # , where="mid")
plt.show(block=False)

Now we will create a cross-section along a row (x-direction).

In [ ]:
section = nc_ds.ugrid.sel(y=41250)
section["head_l1"].isel(time=0).plot.step(x="mesh_x", where="mid")
plt.show(block=False)